In [1]:
%matplotlib inline

import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import tensorflow as tf
tf.keras.backend.set_floatx('float64')

# Deep Neural Network
---
As a neural network goes deeper and deeper, theoretically, it would perform better at least on the training set, but in practice, the performance does not go better with deeper network. One major reason is the `gradient vanishing` problem, in the backpropagation of a very deep network, the gradients at earlier layers would go to zero quickly that cause the learning process to be unbearably slow.

# ResNet
---
ResNet makes it possible, thoretically, to build a infinite deep neural network without impairing the model performance, at least not getting worse. The major advanced structure in `Resnet` is called `skip connection`.

<img src="images/images/skip_connection_kiank.png" style="width:650px;height:200px;">

The left side is a plain network without `skip connection`, the right side is the one with a `skip connection`. 

`skip connection` allows activation values in earlier layers to be fast-forwarded to latter layers by addition. These has 2 advantages:

- In the forward propagation, the latter layer would at least have the performance of earlier layers:

$$ a^{[l + 2]} = g(z^{[l+2]} + a^{[l]})$$

Where $a^{[l]}$ is fast-forwarded from earlier layer. In the iteration process, as weights go to zero, $z$ would decrease to zero, in the worst scenario, even when $z$ totally lost its effect, activation value $a^{[l+2]}$ would at least have the value of $a^{[l]}$.

- In the backpropagation process, the gradient can be directly backpropagated to earlier layers so that there would be fewer multiplications in the backpropagation, thus accelerate the learning process.

## Identity Block
---
There are 2 sorts of `skip connections` in the convolutional network, the first one is called `identity block`, shown as below:

<img src="images/images/idblock3_kiank.png" style="width:650px;height:150px;">

In this structure the activation value of the earlier layer `x` is directly copied and added to the later layer before send to the `ReLU` activation.

Here're the individual steps.

First component of the main path: 
- The first CONV2D has $F_1$ filters of shape (1,1) and a stride of (1,1). Its padding is "valid" and its name should be `conv_name_base + '2a'`. Use 0 as the seed for the random initialization. 
- The first BatchNorm is normalizing the channels axis.  Its name should be `bn_name_base + '2a'`.
- Then apply the ReLU activation function. This has no name and no hyperparameters. 

Second component of main path:
- The second CONV2D has $F_2$ filters of shape $(f,f)$ and a stride of (1,1). Its padding is "same" and its name should be `conv_name_base + '2b'`. Use 0 as the seed for the random initialization. 
- The second BatchNorm is normalizing the channels axis.  Its name should be `bn_name_base + '2b'`.
- Then apply the ReLU activation function. This has no name and no hyperparameters. 

Third component of main path:
- The third CONV2D has $F_3$ filters of shape (1,1) and a stride of (1,1). Its padding is "valid" and its name should be `conv_name_base + '2c'`. Use 0 as the seed for the random initialization. 
- The third BatchNorm is normalizing the channels axis.  Its name should be `bn_name_base + '2c'`. Note that there is no ReLU activation function in this component. 

Final step: 
- The shortcut and the input are added together.
- Then apply the ReLU activation function. This has no name and no hyperparameters. 

In [2]:
def identity_block(X, f, nfilters, stage, block):
    """
    X has shape (m, n_H, n_W, n_C)
    f filter size of second conv + bn + relu
    """
    assert len(nfilters) == 3
    F1, F2, F3 = nfilters
    
    # note: for identity block, the last layer of input X need to has the same dimension of last filter
    # to enable Addition in the end
    assert F3 == X.shape[3]
    conv_base_name = 'conv_' + str(stage) + block
    bn_base_name = 'bn_' + str(stage) + block
    
    X_shortcut = X
    # first
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), 
               padding='valid', kernel_initializer='glorot_uniform',
               name=conv_base_name + '2a')(X)
    X = BatchNormalization(axis=3, name=bn_base_name + '2a')(X)
    X = Activation('relu')(X)
    
    # second
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), 
               padding='same', kernel_initializer='glorot_uniform',
               name=conv_base_name + '2b')(X)
    X = BatchNormalization(axis=3, name=bn_base_name + '2b')(X)
    X = Activation('relu')(X)
    
    # third
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), 
               padding='valid', kernel_initializer='glorot_uniform',
               name=conv_base_name + '2c')(X)
    X = BatchNormalization(axis=3, name=bn_base_name + '2c')(X)  # X shape: (m, n_H, n_W, F3)
    
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [3]:
X = np.random.randn(3, 4, 4, 6)

X_output = identity_block(X, f=2, nfilters=[2, 4, 6], stage='1', block='a')

print('output shape', X_output.shape)

output shape (3, 4, 4, 6)


## Convolutional Block
---
Notice that in the `Identity Block` the shape of input $X$ must equal that of the output of the last convolutional layer so that $X$ and $X_{shortcut}$ could add up. So what if the shape doesn't match up? 

The `convolutional block` help to solve the issue stated above by adding an extra convolutional layer to the shortcut input.

<img src="images/images/convblock_kiank.png" style="width:650px;height:150px;">

The CONV2D layer in the shortcut path is used to resize the input $x$ to a different dimension so that the dimensions match up in the final addition. (This plays a similar role as the matrix $W_s$ discussed in lecture.) For example, to reduce the activation dimensions's height and width by a factor of 2, you can use a 1x1 convolution with a stride of 2. The CONV2D layer on the shortcut path does not use any non-linear activation function. __Its main role is to just apply a (learned) linear function that reduces the dimension of the input, so that the dimensions match up for the later addition step.__

The details of the convolutional block are as follows. 

First component of the main path:
- The first CONV2D has $F_1$ filters of shape (1,1) and a stride of (s,s). Its padding is "valid" and its name should be `conv_name_base + '2a'`. 
- The first BatchNorm is normalizing the channels axis.  Its name should be `bn_name_base + '2a'`.
- Then apply the ReLU activation function. This has no name and no hyperparameters. 

Second component of the main path:
- The second CONV2D has $F_2$ filters of (f,f) and a stride of (1,1). Its padding is "same" and it's name should be `conv_name_base + '2b'`.
- The second BatchNorm is normalizing the channels axis.  Its name should be `bn_name_base + '2b'`.
- Then apply the ReLU activation function. This has no name and no hyperparameters. 

Third component of the main path:
- The third CONV2D has $F_3$ filters of (1,1) and a stride of (1,1). Its padding is "valid" and it's name should be `conv_name_base + '2c'`.
- The third BatchNorm is normalizing the channels axis.  Its name should be `bn_name_base + '2c'`. Note that there is no ReLU activation function in this component. 

Shortcut path:
- The CONV2D has $F_3$ filters of shape (1,1) and a stride of (s,s). Its padding is "valid" and its name should be `conv_name_base + '1'`.
- The BatchNorm is normalizing the channels axis.  Its name should be `bn_name_base + '1'`. 

Final step: 
- The shortcut and the main path values are added together.
- Then apply the ReLU activation function. This has no name and no hyperparameters. 
    


In [4]:
def convolutional_block(X, f, nfilters, s, stage, block):
    """
    X: shape of (m, n_H, n_W, n_C)
    f: filter size of the second convolutional layer
    s: stride size
    """
    
    assert len(nfilters) == 3
    
    conv_base_name = 'conv_' + str(stage) + block
    bn_base_name = 'bn_' + str(stage) + block
    
    F1, F2, F3 = nfilters
    X_shortcut = X
    
    # first
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), 
               padding='valid', name=conv_base_name + '2a', kernel_initializer='glorot_uniform')(X)
    X = BatchNormalization(axis=3, name=bn_base_name + '2a')(X)
    X = Activation('relu')(X)  # (n_H - 1)/s + 1
    
    # second
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), 
               padding='same', name=conv_base_name + '2b', kernel_initializer='glorot_uniform')(X)
    X = BatchNormalization(axis=3, name=bn_base_name + '2b')(X)
    X = Activation('relu')(X)
    
    # third
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), 
               padding='valid', name=conv_base_name + '2c', kernel_initializer='glorot_uniform')(X)
    X = BatchNormalization(axis=3, name=bn_base_name + '2c')(X)
    
    # shortcut
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s),
               padding='valid', name=conv_base_name + '1', kernel_initializer='glorot_uniform')(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_base_name + '1')(X_shortcut)
    
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [5]:
X = np.random.randn(3, 4, 4, 6)

X_output = convolutional_block(X, f=2, nfilters=[2, 4, 6], s=2, stage='1', block='a')

print('output shape', X_output.shape)

output shape (3, 2, 2, 6)


## Building ResNet model (50 layers)
---
You now have the necessary blocks to build a very deep ResNet. The following figure describes in detail the architecture of this neural network. "ID BLOCK" in the diagram stands for "Identity block," and "ID BLOCK x3" means you should stack 3 identity blocks together.

<img src="images/images/resnet_kiank.png" style="width:850px;height:150px;">

The details of this ResNet-50 model are:
- Zero-padding pads the input with a pad of (3,3)
- Stage 1:
    - The 2D Convolution has 64 filters of shape (7,7) and uses a stride of (2,2). Its name is "conv1".
    - BatchNorm is applied to the channels axis of the input.
    - Activation is `relu`.
    - MaxPooling uses a (3,3) window and a (2,2) stride.
- Stage 2:
    - The convolutional block uses three set of filters of size [64,64,256], "f" is 3, "s" is 1 and the block is "a".
    - The 2 identity blocks use three set of filters of size [64,64,256], "f" is 3 and the blocks are "b" and "c".
- Stage 3:
    - The convolutional block uses three set of filters of size [128,128,512], "f" is 3, "s" is 2 and the block is "a".
    - The 3 identity blocks use three set of filters of size [128,128,512], "f" is 3 and the blocks are "b", "c" and "d".
- Stage 4:
    - The convolutional block uses three set of filters of size [256, 256, 1024], "f" is 3, "s" is 2 and the block is "a".
    - The 5 identity blocks use three set of filters of size [256, 256, 1024], "f" is 3 and the blocks are "b", "c", "d", "e" and "f".
- Stage 5:
    - The convolutional block uses three set of filters of size [512, 512, 2048], "f" is 3, "s" is 2 and the block is "a".
    - The 2 identity blocks use three set of filters of size [256, 256, 2048], "f" is 3 and the blocks are "b" and "c".
- The 2D Average Pooling uses a window of shape (2,2) and its name is "avg_pool".
- The flatten doesn't have any hyperparameters or name.
- The Fully Connected (Dense) layer reduces its input to the number of classes using a softmax activation. Its name should be `'fc' + str(classes)`.


In [6]:
def resnet50(input_shape=(64, 64, 3), classes=6):
    
    X_input = Input(input_shape)
    
    X = ZeroPadding2D((3, 3))(X_input)
    
    # stage 1
    X = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), name='conv1', kernel_initializer='glorot_uniform')(X)
    X = BatchNormalization(axis=3, name='bn1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    # stage 2
    X = convolutional_block(X, nfilters=[64, 64, 256], f=3, s=1, stage=2, block='a')
    X = identity_block(X, nfilters=[64, 64, 256], f=3, stage=2, block='b')
    X = identity_block(X, nfilters=[64, 64, 256], f=3, stage=2, block='c')
    
    # stage 3
    X = convolutional_block(X, nfilters=[126, 126, 512], f=3, s=2, stage=3, block='a')
    X = identity_block(X, nfilters=[126, 126, 512], f=3, stage=3, block='b')
    X = identity_block(X, nfilters=[126, 126, 512], f=3, stage=3, block='c')
    X = identity_block(X, nfilters=[126, 126, 512], f=3, stage=3, block='d')
    
    # stage 4
    X = convolutional_block(X, nfilters=[256, 256, 1024], f=3, s=2, stage=4, block='a')
    X = identity_block(X, nfilters=[256, 256, 1024], f=3, stage=4, block='b')
    X = identity_block(X, nfilters=[256, 256, 1024], f=3, stage=4, block='c')
    X = identity_block(X, nfilters=[256, 256, 1024], f=3, stage=4, block='d')
    X = identity_block(X, nfilters=[256, 256, 1024], f=3, stage=4, block='e')
    X = identity_block(X, nfilters=[256, 256, 1024], f=3, stage=4, block='f')
    
    # stage 5
    X = convolutional_block(X, nfilters=[512, 512, 2048], f=3, s=1, stage=5, block='a')
    X = identity_block(X, nfilters=[512, 512, 2048], f=3, stage=5, block='b')
    X = identity_block(X, nfilters=[512, 512, 2048], f=3, stage=5, block='c')
    
    X = AveragePooling2D((2, 2), name='avg_pool')(X)
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer='glorot_uniform')(X)
    
    model = Model(X_input, X, name='resnet50')
    return model

In [7]:
model = resnet50(input_shape=(64, 64, 3), classes=6)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Visualize Image

In [8]:
from tensorflow.keras.utils import plot_model

In [9]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 32, 32, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________